In [16]:
from person_features import PersonFeatures
from global_features import GlobalFeatures
from fusion_module import FusionModule
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from pygame import mixer
import tensorflow as tf
from tqdm import tqdm
from PIL import Image
import pandas as pd
import numpy as np
import scipy.io
import cv2

test_image = "0n5sa6o2zfrg3shtoo.jpg"

In [17]:
# create a dictionary of annotations structures
# hashed by file name
# length of value is number of persons annotated
# [['person'],['gender'],['annotations_categories'],['annotations_continuous'],['gender'],['age']]
emotic_annotations = {}
def create_emotic_dict(data, test= False ): 
    # set variables based on testing/training dataset 
    # using combined avg on test dataset because 
    # multiple annotators used 
    if test: 
        categories = 'combined_categories'
        continuous  = 'combined_continuous'
    else: 
        categories = 'annotations_categories'
        continuous  = 'annotations_continuous'
    
    
    test_image = 'framesdb/images/frame_ghkq7yp0itqz0kn7.jpg'
    emotic_annotations = {} 
    
    print ("Creating body boxes")
    for i in range(len(data['filename'][0])): 
        file = data['filename'][0][i][0]
        # if more than one person is the focus of the image 
        for j in range(len(data['person'][0][i][0])):      
            file_path = str(data['folder'][0][i][0]) + "/" + file
            
            if file_path in emotic_annotations: 
                emotic_annotations[file_path] = np.append( emotic_annotations[file_path], 
                                                [[tuple(np.array(data['person'][0][i][0][j]['body_bbox'][0]).tolist()),
                                                  np.array(np.array(np.array(data['person'][0][i][0][j][categories]).tolist()).tolist()).flatten().tolist(),      
                                                  np.array(np.array(data['person'][0][i][0][j][continuous]).tolist()).flatten().tolist(),
                                                  np.array(data['person'][0][i][0][j]['gender']).tolist(),
                                                  np.array(data['person'][0][i][0][j]['age']).tolist()]], 
                                                  axis = 0 )

            else: 
                emotic_annotations[file_path]= np.array([[tuple(np.array(data['person'][0][i][0][j]['body_bbox'][0]).tolist()),
                                               np.array(np.array(np.array(data['person'][0][i][0][j][categories]).tolist()).tolist()).flatten().tolist(),                             
                                               np.array(np.array(data['person'][0][i][0][j][continuous]).tolist()).flatten().tolist(),
                                               np.array(data['person'][0][i][0][j]['gender']).tolist(),
                                               np.array(data['person'][0][i][0][j]['age']).tolist()]])
    
    return emotic_annotations



In [18]:
# load in emotic annotations
# Annotations = {train, val, test} 
# filename, folder, image_size, original_database, person 
# person: body_bbox, annotations_categories, annotations_continuous, gender, age
print ("Reading in emotic annotations")
emotic = scipy.io.loadmat('EMOTIC/annotations/Annotations.mat')

# dictionary for training and test annotations 
# [['person'],['gender'],['annotations_categories'],['annotations_continuous'],['gender'],['age']]
train_dict = create_emotic_dict(emotic['train'])
val_dict   = create_emotic_dict(emotic['val'], True)
test_dict  = create_emotic_dict(emotic['test'], True)



Reading in emotic annotations
Creating body boxes
Creating body boxes
Creating body boxes


In [19]:
import os
test_dir = 'prototype_imgs/'


test_files = ["mscoco/images/"+i for i in os.listdir(test_dir) if 'jpg' in i]
print(test_files)

['mscoco/images/COCO_train2014_000000001424.jpg', 'mscoco/images/COCO_train2014_000000416690.jpg', 'mscoco/images/COCO_train2014_000000416651.jpg', 'mscoco/images/COCO_train2014_000000073568.jpg', 'mscoco/images/COCO_train2014_000000020651.jpg', 'mscoco/images/COCO_train2014_000000076992.jpg', 'mscoco/images/COCO_train2014_000000572179.jpg', 'mscoco/images/COCO_train2014_000000569401.jpg']


In [20]:
photo_avgs = {} 
for k,v in train_dict.items(): 
    if k in test_files:
        mean_v = 0 
        mean_a = 0 

        for i in v: 
            mean_v += i[2][0]
            mean_a +=i[2][1]
            
        photo_avgs[k] = (mean_v/len(v), mean_a/len(v))
   
count = 0
for k,v in photo_avgs.items(): 
    count += 1

    print (count)
    print(k,v)
    print (train_dict[k][0])
    


1
mscoco/images/COCO_train2014_000000020651.jpg (9.5, 7.0)
[(357, 5, 594, 359) list(['Anticipation', 'Excitement', 'Happiness'])
 list([10, 7, 6]) list(['Female']) list(['Teenager'])]
2
mscoco/images/COCO_train2014_000000416690.jpg (6.0, 8.0)
[(175, 109, 420, 472) list(['Excitement', 'Happiness', 'Surprise'])
 list([6, 8, 8]) list(['Male']) list(['Kid'])]
3
mscoco/images/COCO_train2014_000000001424.jpg (5.0, 4.0)
[(36, 191, 195, 370) list(['Disconnection', 'Doubt/Confusion'])
 list([5, 4, 2]) list(['Female']) list(['Kid'])]
4
mscoco/images/COCO_train2014_000000416651.jpg (5.0, 5.0)
[(271, 63, 401, 516) list(['Engagement']) list([5, 5, 7]) list(['Male'])
 list(['Adult'])]
5
mscoco/images/COCO_train2014_000000076992.jpg (8.0, 7.0)
[(19, 55, 328, 327) list(['Anticipation', 'Engagement']) list([8, 7, 7])
 list(['Male']) list(['Adult'])]
6
mscoco/images/COCO_train2014_000000572179.jpg (6.0, 5.5)
[(297, 27, 548, 308) list(['Anticipation', 'Engagement']) list([6, 7, 7])
 list(['Male']) list([

In [21]:
emotions = ['Peace', 'Affection', 'Esteem', 'Anticipation', 'Engagement','Confidence','Happiness','Pleasure','Excitement','Surprise','Sympathy','Doubt/Confusion','Disconnection','Fatigue','Embarrassment','Yearning','Disapproval','Aversion','Annoyance','Anger','Sensitivity','Sadness','Disquietment','Fear','Pain','Suffering']
basic = [emotions[6], emotions[9], emotions[17], emotions[19], emotions[21], emotions[23]]
 


In [22]:
from sklearn import neighbors 

songs_csv = 'DEAM/annotations/annotations_avg_per_song/song_level/26_category_emotions.csv'
songs_df = pd.read_csv(songs_csv)
songs_df.sort_values(by=['emotion assignment','song_id'], inplace=True, ascending=True)
songs_df = songs_df.reset_index()

X = songs_df[['valence_mean', 'arousal_mean']]
y = songs_df['emotion assignment']

clf = neighbors.KNeighborsClassifier(n_neighbors=3, weights='distance')  
clf.fit(X,y) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='distance')

In [8]:
# import numpy as np
# from sklearn.neighbors import KDTree

# X = np.array(songs_df[['valence_mean', 'arousal_mean']])  # 3 points in 2 dimensions
# tree = KDTree(X)
# dist, ind = tree.query([[9.5, 7.0]], k=5)
# print(ind[0][0])  # indices of 2 closest neighbors
# # print(dist)  # distances to 2 closest neighbors
# x = songs_df['emotion assignment'].iloc[ind[0][0]]

# print (emotions[x])


In [9]:
videos_emot = [] 
for k,v in photo_avgs.items(): 
#     print(k)
    emot= clf.predict([v])[0]
    neigh=clf.kneighbors(X, return_distance=False) 
    videos_emot.append(emot)
#     print(emot)
#     print(emotions[emot])
     
print (videos_emot)
    

[5, 23, 25, 17, 18, 16, 25, 4]


In [10]:
# songs_df.sort_values(by=['emotion assignment','song_id'], inplace=True, ascending=True)
# songs_df = songs_df.reset_index()

In [11]:

for i, k in enumerate(photo_avgs.keys()): 
    # play emotion-tagged music 
    mixer.init()
    music_dir = 'DEAM/MEMD_audio/'
    # select music by valence and arousal scores! 
    emot = videos_emot[i]
    df = songs_df['song_id'].loc[songs_df['emotion assignment'] == emot]
    song_id= df.sample(n=1).iloc[0]
#     print (k)
#     img = cv2.imread(k) 
#     cv2.imshow(img, 1)
#     cv2.waitKey(0)
    
    mixer.music.load(music_dir + str(song_id) + ".mp3")
    mixer.music.play()

In [12]:
# import eyed3

# audiofile = eyed3.load('example.mp3')
# if (audiofile.tag == None):
#     audiofile.initTag()

# audiofile.tag.images.set(3, open('cover.jpg','rb').read(), 'image/jpeg')

# audiofile.tag.save()

In [13]:
# from keras.preprocessing import image
# from keras.applications.vgg16 import VGG16
# from keras.applications.vgg16 import preprocess_input
# from keras.utils.data_utils import get_file
# train_filelist = np.array(train_dict.keys()).tolist()
# dir_path = "EMOTIC/emotic/"

# proccesed_ims = [] 
# for i in tqdm(train_filelist): 
#     img_path = dir_path + i 
#     img = image.load_img(img_path, target_size=(224, 224))
#     # convert image to numpy array
#     x = image.img_to_array(img)
#     # the image is now in an array of shape (3, 224, 224) 
#     # need to expand it to (1, 3, 224, 224) as preprocess takes in a list
#     x = np.expand_dims(x, axis=0)
#     x = preprocess_input(x)
#     proccesed_ims.append(x)
    
# x = np.array(proccesed_ims)
# x.dump("train_imgs.npy")
# print ("DUMPED")


In [14]:
# proccesed_ims_test = [] 
# test_filelist = np.array(test_dict.keys()).tolist()

# for i in tqdm(test_filelist): 
#     img_path = dir_path + i 
#     img = image.load_img(img_path, target_size=(224, 224))
#     # convert image to numpy array
#     x = image.img_to_array(img)
#     # the image is now in an array of shape (3, 224, 224) 
#     # need to expand it to (1, 3, 224, 224) as preprocess takes in a list
#     x = np.expand_dims(x, axis=0)
#     x = preprocess_input(x)
#     proccesed_ims_test.append(x)
    
# x = np.array(proccesed_ims_test)
# x.dump("test_imgs.npy")

In [15]:
# # for k,v in emotic_annotations.items(): 
# #     print(k, v)
    
# # pass bounding box parameters to person to handle
# person_features = PersonFeatures(train_dict, test_dict)

# global_features = GlobalFeatures(train_dict['train']['filename'])
# print (global_features)
# # fusion = GlobalFeatures(annotations['train']['filename'])

